In [3]:
#Dumb charging
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *


Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 22:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def parameterdc( M, T, IE,RE,TS,TD):

    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
        #print("StartTime:" , TS[i])
        #print("DepartureTime:" , TD[i])
        #print("Initial energy:" , Eint[i])
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHP_a = float(row[0])
        HHP_b = float(row[1])
        HHP_c = float(row[2])
        HHQ_a = float(row[3])
        HHQ_b = float(row[4])
        HHQ_c = float(row[5])
        HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("Generalizeparam")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    
    #Phase constraint:
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
    
    #power outside TS TD as 0:
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
            
    #Subcost calculation:
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            if p[i,t] >= 0:                     # if power val is +, charging so eff is 0.9
                e[i][t] +=  p[i,t] * 0.9 * 1/4  # energy consumed
            elif p[i,t] < 0:                               # if power val is -, discharging so eff is 1.1
                e[i][t] +=  p[i,t] * 1.1 * 1/4 
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]    
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
        
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    
    #set objective:
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(C)
    
    #writing the model:
    m.write('dumbcharging.lp')
    
    # Compute optimal solution:
    m.optimize()
    m.write('dumbcharge.sol')
    #print solution: 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    finalcost = m.objval
    return(finalcost);

def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EV1sampledata.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
M=2
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
#Ev[0] = ev1arrdeptime()
#Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[0] = '01.01.2014 16:45:00'
    Starttime[1] = '01.01.2014 18:00:00'
    Departtime[0] = '01.01.2014 21:00:00'
    Departtime[1] = '01.01.2014 23:30:00'
    IE[0] = RE - 5.95
    IE[1] = RE - 5.95
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]] 
    IE =[IE[0],IE[1]]
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    IE =[IE[1],IE[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)

if TD[0] > TS[1] and TD[0] < 96 and TD[1] > TS[1] and TD[1] < 96:
    if TD[0] < TD[1] or TD[1] < TD[0] or TD[1] == TD[0]:
        print('case 1: Both EVs depart before mid-night')
        Start = [TS[0],TS[1]]
        Depart = [TD[0],TD[1]]
        cost1=parameterdc(2,96,IE,RE,Start,Depart)
        with open('dumbcharge.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('dumbchargecase1.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()      
        finalcost = cost1
        print('final optimal cost:',finalcost)
    
elif TD[0] < TS[0] and TD[0] >= 0 and TD[1] < TS[0] and TD[1] >= 0:
    if TD[0] < TD[1] or TD[1] < TD[0]:
        print('case2: Both EVs depart in the morning')
        Start = [TS[0],TS[1]]
        Depart = [96,96]
        cost1=parameterdc(2,96,IE,RE,Start,Depart)
        with open('dumbcharge.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('dumbchargecase2.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        with open('dumbchargecase2.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],96):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                if p[i] >= 0:
                    pwr[i] =  p[i] * 0.9 * 1/4  # 
                else:
                    pwr[i] =  p[i] * 1.1 * 1/4  
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            powr =[]
            po = []
            for row in csv_reader:
                powr_str = row[0]
                powr_P = float(row[1])
                powr.append([powr_str,powr_P])
            for i in range(len(powr)):
                for j in range(TS[1],96):
                    if powr[i][0] == 'Power[%d,%d]' %(1,j):
                        po.append(powr[i][1])
            print(po)
            egy =[]
            pwrs=[0 for i in range(T)]
            for i in range(len(po)):
                if po[i] >= 0:
                    pwrs[i] =  po[i] * 0.9 * 1/4  # 
                else:
                    pwrs[i] =  po[i] * 1.1 * 1/4 
                egy.append(pwrs[i])
                IEgy[1] += egy[i]
            print(egy)
            print(IEgy[1])
        Start = [0,0]
        Depart = [TD[0],TD[1]]
        IE[0] += InitialEgy[0]
        IE[1] += IEgy[1]
        cost2=parameterdc(2,96,IE,RE,Start,Depart)
        with open('dumbcharge.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('dumbchargecase2.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        f.close()
        myfile.close()
        finalcost = cost1 + cost2 
        print('final optimal cost:',finalcost)
elif TD[0] > TS[1] and TD[0] < 96 and TD[1] < TS[0] and TD[1] >= 0:
    print('case3: EV1 depart before mid-night and EV2 departs in the morning')
    Start = [TS[0],TS[1]]
    Depart = [TD[0],96]
    cost1=parameterdc(2,96,IE,RE,Start,Depart)
    with open('dumbcharge.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dumbchargecase3.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    with open('dumbchargecase3.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        powr =[]
        po = []
        for row in csv_reader:
            powr_str = row[0]
            powr_P = float(row[1])
            powr.append([powr_str,powr_P])
        for i in range(len(powr)):
            for j in range(TS[1],96):
                if powr[i][0] == 'Power[%d,%d]' %(1,j):
                    po.append(powr[i][1])
        print(po)
        egy =[]
        pwrs=[0 for i in range(T)]
        for i in range(len(po)):
            if po[i] >= 0:
                pwrs[i] =  po[i] * 0.9 * 1/4  # 
            else:
                pwrs[i] =  po[i] * 1.1 * 1/4 
            egy.append(pwrs[i])
            IEgy[1] += egy[i]
        print(egy)
        print(IEgy[1])
    Start = [0]
    Depart = [TD[1]]
    IE[1] += IEgy[1]
    cost2=parameterdc(1,96,IE,RE,Start,Depart)
    with open('dumbcharge.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dumbchargecase3.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    f.close()
    myfile.close()
    finalcost = cost1 + cost2 
    print('final optimal cost:',finalcost)      
elif TD[1] > TS[1] and TD[1] < 96 and TD[0] < TS[0] and TD[0] >= 0:
    print('case4: EV2 depart before mid-night and EV1 departs in the morning')
    Start = [TS[0],TS[1]]
    Depart = [96,TD[1]]
    cost1=parameterdc(2,96,IE,RE,Start,Depart)
    with open('dumbcharge.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dumbchargecase4.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    with open('dumbchargecase4.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[0],96):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        InitialEgy[0] =0
        pwr=[0 for i in range(T)]
        for i in range(len(p)):
            if p[i] >= 0:
                pwr[i] =  p[i] * 0.9 * 1/4  # 
            else:
                pwr[i] =  p[i] * 1.1 * 1/4  
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])
    Start = [0]
    Depart = [TD[0]]
    print('IE before', IE[0])
    IE[0] += InitialEgy[0]
    print('IE after', IE[0])
    cost2=parameterdc(1,96,IE,RE,Start,Depart)
    with open('dumbcharge.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dumbchargecase4.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    f.close()
    myfile.close()
    finalcost = cost1 + cost2  
    print('final optimal cost:',finalcost) 
else:
    exit

# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)

Initial energy: 11.05
ev_a: 01.01.2014 16:45:00
ev_d: 01.01.2014 21:00:00
Initial energy: 11.05
ev_a: 01.01.2014 18:00:00
ev_d: 01.01.2014 23:30:00
ev1 arrives before
[67, 72]
[84, 94]
case 1: Both EVs depart before mid-night
Optimize a model with 351 rows, 768 columns and 846 nonzeros
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [7e-02, 7e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.7290356
Presolve removed 351 rows and 768 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 16 available processors)

Solution count 2: 4.09326 3.72904 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.093262754000e+00, best bound 4.093262754000e+00, gap 0.0000%
a[0,0] 1.0
a[0,1] 1.0
a[0,2] 1.0
a[0,3] 1.0
a[0,4] 1.0
a[0,5] 1.0
a[0,6] 1.0
a[0,7] 

final optimal cost: 4.093262753999998


In [2]:
#dayahead scheduling
#!/usr/bin/python
import csv
import numpy as np
from datetime import time,datetime,date
from scipy import interpolate
from gurobipy import *
import random

Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 22:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def parameterda( M, T, IE,RE,TS,TD):

    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHDate = row[0]
        HHP_a = float(row[1])
        HHP_b = float(row[2])
        HHP_c = float(row[3])
        HHQ_a = float(row[4])
        HHQ_b = float(row[5])
        HHQ_c = float(row[6])
        HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_Scheduling.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PVDate = row[0]
        PV_P = float(row[1])
        PV.append([PVDate,PV_P])
    m = Model("dayahead")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            if p[i,t] >= 0:                     # if power val is +, charging so eff is 0.9
                e[i][t] +=  p[i,t] * 0.9 * 1/4  # energy consumed
            elif p[i,t] < 0:                               # if power val is -, discharging so eff is 1.1
                e[i][t] +=  p[i,t] * 1.1 * 1/4  
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    ST= STime[0]
    print(len(STime))
    print(len(STime))
    if len(DTime)== 1:
        DT = DTime[0]
    else:
        if DTime[0]>=DTime[1]:
            DT = DTime[0]
        elif DTime[1]>=DTime[0]:
            DT = DTime[1]
    for t in range(ST,DT):
        for i in range(M):
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]

        pahhpv[t] = patot[t] + HH[t][1] - PV[t][1]
        pbhhpv[t] = pbtot[t] + HH[t][2]
        pchhpv[t] = pctot[t] + HH[t][3]
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('dayahead.lp')

    # Compute optimal solution
    m.optimize()
    m.write('dayahead.sol')
    
    #print solution 
    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    finalcost = m.objVal

# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)
    return(finalcost);
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
M = 2
T = 96
InitialEgy = [0 for i in range(M)]
IEgy = [0 for i in range(M)]
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[0] = '01.01.2014 16:45:00'
    Starttime[1] = '01.01.2014 18:00:00'
    Departtime[0] = '01.01.2014 21:00:00'
    Departtime[1] = '01.01.2014 23:30:00'
    IE[0] = RE - 5.95
    IE[1] = RE - 5.95
    #Starttime[i] = Date + Ev[i][1] +Time
    #Departtime[i] = Date + Ev[i][2] +Time
    #print('energy:',Ev[i][0])
    #IE[i] = RE - Ev[i][0]
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]] 
    IE =[IE[0],IE[1]]
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    IE =[IE[1],IE[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)

if TD[0] > TS[1] and TD[0] < 96 and TD[1] > TS[1] and TD[1] < 96:
    if TD[0] < TD[1] or TD[1] < TD[0] or TD[1] == TD[0]:
        print('case 1: Both EVs depart before mid-night')
        Start = [TS[0],TS[1]]
        Depart = [TD[0],TD[1]]
        cost1=parameterda(2,96,IE,RE,Start,Depart)
        with open('dayahead.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('dayaheadcase1.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()      
        finalcost = cost1
        print('final optimal cost:',finalcost)
    
elif TD[0] < TS[0] and TD[0] >= 0 and TD[1] < TS[0] and TD[1] >= 0:
    if TD[0] < TD[1] or TD[1] < TD[0]:
        print('case2: Both EVs depart in the morning')
        Start = [TS[0],TS[1]]
        Depart = [96,96]
        cost1=parameterda(2,96,IE,RE,Start,Depart)
        with open('dayahead.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('dayaheadcase2.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        with open('dayaheadcase2.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],96):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            InitialEgy[0] =0
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                if p[i] >= 0:
                    pwr[i] =  p[i] * 0.9 * 1/4  # 
                else:
                    pwr[i] =  p[i] * 1.1 * 1/4  
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            powr =[]
            po = []
            for row in csv_reader:
                powr_str = row[0]
                powr_P = float(row[1])
                powr.append([powr_str,powr_P])
            for i in range(len(powr)):
                for j in range(TS[1],96):
                    if powr[i][0] == 'Power[%d,%d]' %(1,j):
                        po.append(powr[i][1])
            print(po)
            egy =[]
            IEgy[1] =0
            pwrs=[0 for i in range(T)]
            for i in range(len(po)):
                if po[i] >= 0:
                    pwrs[i] =  po[i] * 0.9 * 1/4  # 
                else:
                    pwrs[i] =  po[i] * 1.1 * 1/4 
                egy.append(pwrs[i])
                IEgy[1] += egy[i]
            print(egy)
            print(IEgy[1])
        Start = [0,0]
        Depart = [TD[0],TD[1]]
        IE[0] += InitialEgy[0]
        IE[1] += IEgy[1]
        cost2=parameterda(2,96,IE,RE,Start,Depart)
        with open('dayahead.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('dayaheadcase2.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        f.close()
        myfile.close()
        finalcost = cost1 + cost2 
        print('final optimal cost:',finalcost)
elif TD[0] > TS[1] and TD[0] < 96 and TD[1] < TS[0] and TD[1] >= 0:
    print('case3: EV1 depart before mid-night and EV2 departs in the morning')
    Start = [TS[0],TS[1]]
    Depart = [TD[0],96]
    cost1=parameterda(2,96,IE,RE,Start,Depart)
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayaheadcase3.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    with open('dayaheadcase3.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        powr =[]
        po = []        
        for row in csv_reader:
            powr_str = row[0]
            powr_P = float(row[1])
            powr.append([powr_str,powr_P])
        for i in range(len(powr)):
            for j in range(TS[1],96):
                if powr[i][0] == 'Power[%d,%d]' %(1,j):
                    po.append(powr[i][1])
        print(po)
        egy =[]
        IEgy[1] =0
        pwrs=[0 for i in range(T)]
        for i in range(len(po)):
            if po[i] >= 0:
                pwrs[i] =  po[i] * 0.9 * 1/4  # 
            else:
                pwrs[i] =  po[i] * 1.1 * 1/4 
            egy.append(pwrs[i])
            IEgy[1] += egy[i]
        print(egy)
        print(IEgy[1])
    Start = [0]
    Depart = [TD[1]]
    IE[1] += IEgy[1]
    cost2=parameterda(1,96,IE,RE,Start,Depart)
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayaheadcase3.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    f.close()
    myfile.close()
    finalcost = cost1 + cost2 
    print('final optimal cost:',finalcost)      
elif TD[1] > TS[1] and TD[1] < 96 and TD[0] < TS[0] and TD[0] >= 0:
    print('case4: EV2 depart before mid-night and EV1 departs in the morning')
    Start = [TS[0],TS[1]]
    Depart = [96,TD[1]]
    cost1=parameterda(2,96,IE,RE,Start,Depart)
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayaheadcase4.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    with open('dayaheadcase4.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[0],96):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        InitialEgy[0] =0
        pwr=[0 for i in range(T)]
        for i in range(len(p)):
            if p[i] >= 0:
                pwr[i] =  p[i] * 0.9 * 1/4  # 
            else:
                pwr[i] =  p[i] * 1.1 * 1/4  
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])
    Start = [0]
    Depart = [TD[0]]
    IE[0] += InitialEgy[0]
    cost2=parameterda(1,96,IE,RE,Start,Depart)
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayaheadcase4.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    f.close()
    myfile.close()
    finalcost = cost1 + cost2  
    print('final optimal cost:',finalcost) 
else:
    exit

Initial energy: 11.05
ev_a: 01.01.2014 16:45:00
ev_d: 01.01.2014 21:00:00
Initial energy: 11.05
ev_a: 01.01.2014 18:00:00
ev_d: 01.01.2014 23:30:00
ev1 arrives before
[67, 72]
[84, 94]
case 1: Both EVs depart before mid-night
2
2
Optimize a model with 351 rows, 768 columns and 846 nonzeros
Model has 162 quadratic constraints
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 7e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 310 rows and 612 columns
Presolve time: 0.02s
Presolved: 905 rows, 624 columns, 2730 nonzeros
Variable types: 507 continuous, 117 integer (117 binary)

Root relaxation: objective 3.457091e+00, 336 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Tim

Power[0,74] 4.9339998115002315
Power[0,75] 4.979999811561618
Power[0,76] 2.186124101012571
Power[0,77] 1.889596920754406
Power[0,78] -4.332162982863876
Power[0,79] 2.0851084293136717
Power[0,80] -4.664999638850089
Power[0,81] -4.824999638291022
Power[0,82] -4.862999635721076
Power[0,83] -4.61099964186883
Power[0,84] 0.0
Power[0,85] 0.0
Power[0,86] 0.0
Power[0,87] 0.0
Power[0,88] 0.0
Power[0,89] 0.0
Power[0,90] 0.0
Power[0,91] 0.0
Power[0,92] 0.0
Power[0,93] 0.0
Power[0,94] 0.0
Power[0,95] 0.0
Power[1,0] 0.0
Power[1,1] 0.0
Power[1,2] 0.0
Power[1,3] 0.0
Power[1,4] 0.0
Power[1,5] 0.0
Power[1,6] 0.0
Power[1,7] 0.0
Power[1,8] 0.0
Power[1,9] 0.0
Power[1,10] 0.0
Power[1,11] 0.0
Power[1,12] 0.0
Power[1,13] 0.0
Power[1,14] 0.0
Power[1,15] 0.0
Power[1,16] 0.0
Power[1,17] 0.0
Power[1,18] 0.0
Power[1,19] 0.0
Power[1,20] 0.0
Power[1,21] 0.0
Power[1,22] 0.0
Power[1,23] 0.0
Power[1,24] 0.0
Power[1,25] 0.0
Power[1,26] 0.0
Power[1,27] 0.0
Power[1,28] 0.0
Power[1,29] 0.0
Power[1,30] 0.0
Power[1,31] 0.0


In [1]:
DTime = [27,33]
STime= [33]
if DTime[0]>=DTime[1]:
    print(DTime[0])
#elif DTime[1]>=DTime[0]:
#    print(DTime[1])
elif len(STime)==1:
    ST = STime[0]
    print('ST val:',ST)

ST val: 33


In [1]:
#Test Dynamic Scheduling
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *
import random
import time
import sys

M=2
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 22:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
def parameterdy( i, T, IE,RE,TS,TD):
    M= i
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHDate = row[0]
        HHP_a = float(row[1])
        HHP_b = float(row[2])
        HHP_c = float(row[3])
        HHQ_a = float(row[4])
        HHQ_b = float(row[5])
        HHQ_c = float(row[6])
        HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_Scheduling.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PVDate = row[0]
        PV_P = float(row[1])
        PV.append([PVDate,PV_P])
    m = Model("dynamicscheduling")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            if p[i,t] >= 0:
                e[i][t] +=  p[i,t] * 0.9 * 1/4  # 
            elif p[i,t] < 0:
                e[i][t] +=  p[i,t] * 1.1 * 1/4
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    ST= STime[0]
    print(len(STime))
    print(len(DTime))
    if len(DTime)== 1:
        DT = DTime[0]
    else:
        if DTime[0]>=DTime[1]:
            DT = DTime[0]
        elif DTime[1]>=DTime[0]:
            DT = DTime[1]
    for t in range(ST,DT):
        for i in range(M):

            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]

        pahhpv[t] = patot[t] + HH[t][1] - PV[t][1]
        pbhhpv[t] = pbtot[t] + HH[t][2]
        pchhpv[t] = pctot[t] + HH[t][3]
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost

    m.setObjective(Im_C)

    m.write('testDynamicscheduling.lp')


    # Compute optimal solution
    m.optimize()
    m.write('testDynamicscheduling.sol')
    
    #print solution 
    for v in m.getVars():
        print(v.varName, v.x)
        
    print('Obj:', m.objVal)
    finalcost = m.objVal
         
    return(finalcost);
M = 2
T = 96
InitialEgy = [0 for i in range(M)]
energycost1 = [0 for i in range(M)]
IEgy = [0 for i in range(M)] 
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[0] = '01.01.2014 16:45:00'
    Starttime[1] = '01.01.2014 18:00:00'
    Departtime[0] = '01.01.2014 21:00:00'
    Departtime[1] = '01.01.2014 23:30:00'
    IE[0] = RE - 5.95
    IE[1] = RE - 5.95
    #Starttime[i] = Date + Ev[i][1] +Time
    #Departtime[i] = Date + Ev[i][2] +Time
    #print('energy:',Ev[i][0])
    #IE[i] = RE - Ev[i][0]
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]]
    IE =[IE[0],IE[1]]
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    IE =[IE[1],IE[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)
if TD[0] > TS[1] and TD[0] < 96 and TD[1] > TS[1] and TD[1] < 96:
    if TD[0] < TD[1] or TD[1] < TD[0] or TD[1] == TD[0]:
        print('case 1: Both EVs depart before mid-night')
        Start = [TS[0]]
        Depart = [TD[0]]
        cost1=parameterdy(1,96,IE,RE,Start,Depart)

        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling1ev.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],TS[1]):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            InitialEgy[0] =0
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                if p[i] >= 0:
                    pwr[i] =  p[i] * 0.9 * 1/4  # 
                else:
                    pwr[i] =  p[i] * 1.1 * 1/4   
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
            file2 = open(path2, newline='')
            reader2 = csv.reader(file2)
            header2 = next(reader2)
            marketcost = []
            for row in reader2 :
                mcost = float(row[0])
                marketcost.append(mcost)
            cost = [j/100 for j in marketcost]
            i = 0
            for t in range(TS[0],TS[1]):
                energycost1[0] += energy[i] * cost[t]
                i +=1
            print(energycost1[0])            
        Start = [TS[1],TS[1]]
        Depart = [TD[0],TD[1]]
        print('IE before', IE[0])
        IE[0] += InitialEgy[0]
        print('IE after', IE[0])
        cost2=parameterdy(2,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()
        finalcost = energycost1[0] + cost2
        print('final optimal cost:',finalcost)
    
elif TD[0] < TS[0] and TD[0] >= 0 and TD[1] < TS[0] and TD[1] >= 0:
    if TD[0] < TD[1] or TD[1] < TD[0]:
        print('case2: Both EVs depart in the morning')
        Start = [TS[0]]
        Depart = [96]
        cost1=parameterdy(1,96,IE,RE,Start,Depart)

        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling1ev.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],TS[1]):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            InitialEgy[0] =0
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                if p[i] >= 0:
                    pwr[i] =  p[i] * 0.9 * 1/4  # 
                else:
                    pwr[i] =  p[i] * 1.1 * 1/4  
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
            file2 = open(path2, newline='')
            reader2 = csv.reader(file2)
            header2 = next(reader2)
            marketcost = []
            for row in reader2 :
                mcost = float(row[0])
                marketcost.append(mcost)
            cost = [j/100 for j in marketcost]    
            i = 0
            for t in range(TS[0],TS[1]):
                energycost1[0] += energy[i] * cost[t]
                i += 1
            print(energycost1[0])            
        Start = [TS[1],TS[1]]
        Depart = [96,96]
        IE[0] += InitialEgy[0]
        cost2=parameterdy(2,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
        with open('testDynamicscheduling2evs.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            powr =[]
            po = []
            for row in csv_reader:
                powr_str = row[0]
                powr_P = float(row[1])
                powr.append([powr_str,powr_P])
            for i in range(len(powr)):
                for j in range(TS[1],96):
                    if powr[i][0] == 'Power[%d,%d]' %(0,j):
                        po.append(powr[i][1])
            print(po)
            energy =[]
            InitialEgy[0] =0
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                if p[i] >= 0:
                    pwr[i] =  p[i] * 0.9 * 1/4  # 
                else:
                    pwr[i] =  p[i] * 1.1 * 1/4  
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            powr =[]
            po = []
            for row in csv_reader:
                powr_str = row[0]
                powr_P = float(row[1])
                powr.append([powr_str,powr_P])
            for i in range(len(powr)):
                for j in range(TS[1],96):
                    if powr[i][0] == 'Power[%d,%d]' %(1,j):
                        po.append(powr[i][1])
            print(po)
            egy =[]
            IEgy[1] =0
            pwrs=[0 for i in range(T)]
            for i in range(len(po)):
                if po[i] >= 0:
                    pwrs[i] =  po[i] * 0.9 * 1/4  # 
                else:
                    pwrs[i] =  po[i] * 1.1 * 1/4 
                egy.append(pwrs[i])
                IEgy[1] += egy[i]
            print(egy)
            print(IEgy[0])           
        Start = [0,0]
        Depart = [TD[0],TD[1]]
        IE[0] += InitialEnergy[0]
        IE[1] += IEgy[1]
        cost3=parameterdy(2,96,IE,RE,Start,Depart) 
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()
        finalcost = energycost1[0] + cost2 + cost3
        print('final optimal cost:',finalcost)
elif TD[0] > TS[1] and TD[0] < 96 and TD[1] < TS[0] and TD[1] >= 0:
    print('case3: EV1 depart before mid-night and EV2 departs in the morning')
    Start = [TS[0]]
    Depart = [TD[0]]
    cost1=parameterdy(1,96,IE,RE,Start,Depart)

    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)            
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    with open('testDynamicscheduling1ev.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[0],TS[1]):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        InitialEgy[0] =0
        pwr=[0 for i in range(T)]
        for i in range(len(p)):
            if p[i] >= 0:
                pwr[i] =  p[i] * 0.9 * 1/4  # 
            else:
                pwr[i] =  p[i] * 1.1 * 1/4 
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])
        path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
        file2 = open(path2, newline='')
        reader2 = csv.reader(file2)
        header2 = next(reader2)
        marketcost = []
        for row in reader2 :
            mcost = float(row[0])
            marketcost.append(mcost)
        cost = [j/100 for j in marketcost]    
        i = 0
        for t in range(TS[0],TS[1]):
            energycost1[0] += energy[i] * cost[t]
            i += 1
        print(energycost1[0])            
    Start = [TS[1],TS[1]]
    Depart = [TD[0],96]
    IE[0] += InitialEgy[0]
    cost2=parameterdy(2,96,IE,RE,Start,Depart)
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    with open('testDynamicscheduling2evs.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        powr =[]
        po = []
        for row in csv_reader:
            powr_str = row[0]
            powr_P = float(row[1])
            powr.append([powr_str,powr_P])
        for i in range(len(powr)):
            for j in range(TS[1],96):
                if powr[i][0] == 'Power[%d,%d]' %(1,j):
                    po.append(powr[i][1])
        print(po)
        egy =[]
        IEgy[1] =0
        pwrs=[0 for i in range(T)]
        for i in range(len(po)):
            if po[i] >= 0:
                pwrs[i] =  po[i] * 0.9 * 1/4  # 
            else:
                pwrs[i] =  po[i] * 1.1 * 1/4 
            egy.append(pwrs[i])
            IEgy[1] += egy[i]
        print(energy)
        print(IEgy[0])           
    Start = [0]
    Depart = [TD[1]]
    IE[1] += IEgy[1]
    cost3=parameterdy(1,96,IE,RE,Start,Depart) 
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    finalcost = energycost1[0] + cost2 + cost3
    print('final optimal cost:',finalcost)       
elif TD[1] > TS[1] and TD[1] < 96 and TD[0] < TS[0] and TD[0] >= 0:
    print('case4: EV2 depart before mid-night and EV1 departs in the morning')
    Start = [TS[0]]
    Depart = [96]
    cost1=parameterdy(1,96,IE,RE,Start,Depart)

    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)            
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    with open('testDynamicscheduling1ev.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[0],TS[1]):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        InitialEgy[0] =0
        pwr=[0 for i in range(T)]
        for i in range(len(p)):
            if p[i] >= 0:
                pwr[i] =  p[i] * 0.9 * 1/4  # 
            else:
                pwr[i] =  p[i] * 1.1 * 1/4             
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])
        path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
        file2 = open(path2, newline='')
        reader2 = csv.reader(file2)
        header2 = next(reader2)
        marketcost = []
        for row in reader2 :
            mcost = float(row[0])
            marketcost.append(mcost)
        cost = [j/100 for j in marketcost]    
        i = 0
        for t in range(TS[0],TS[1]):
            energycost1[0] += energy[i] * cost[t]
            i += 1
        print(energycost1[0])            
    Start = [TS[1],TS[1]]
    Depart = [96,TD[1]]
    IE[0] += InitialEgy[0]
    cost2=parameterdy(2,96,IE,RE,Start,Depart)
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    with open('testDynamicscheduling2evs.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[1],96):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        InitialEgy[0] =0
        pwrs=[0 for i in range(T)]
        for i in range(len(p)):
            if p[i] >= 0:
                pwr[i] =  p[i] * 0.9 * 1/4  # 
            else:
                pwr[i] =  p[i] * 1.1 * 1/4 
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])           
    Start = [0]
    Depart = [TD[0]]
    IE[0] += InitialEgy[0]
    cost3=parameterdy(1,96,IE,RE,Start,Depart) 
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    finalcost = energycost1[0] + cost2 + cost3
    print('final optimal cost:',finalcost)  
else:
    exit

Initial energy: 11.05
ev_a: 01.01.2014 16:45:00
ev_d: 01.01.2014 21:00:00
Initial energy: 11.05
ev_a: 01.01.2014 18:00:00
ev_d: 01.01.2014 23:30:00
ev1 arrives before
[67, 72]
[84, 94]
case 1: Both EVs depart before mid-night
Academic license - for non-commercial use only
1
1
Optimize a model with 178 rows, 384 columns and 418 nonzeros
Model has 102 quadratic constraints
Variable types: 96 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 7e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [4e+00, 6e+00]
Presolve removed 160 rows and 316 columns
Presolve time: 0.01s
Presolved: 426 rows, 272 columns, 1190 nonzeros
Variable types: 221 continuous, 51 integer (51 binary)

Root relaxation: objective 1.777793e+00, 165 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI


Root relaxation: objective 1.940337e+00, 280 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.94034    0   45          -    1.94034      -     -    0s
H    0     0                       2.0470208    1.94034  5.21%     -    0s
H    0     0                       2.0370930    1.94034  4.75%     -    0s
     0     0    1.97653    0   35    2.03709    1.97653  2.97%     -    0s
     0     0    1.98186    0   35    2.03709    1.98186  2.71%     -    0s
H    0     0                       2.0300028    1.98186  2.37%     -    0s
     0     0    1.98530    0   33    2.03000    1.98530  2.20%     -    0s
     0     0    1.98543    0   38    2.03000    1.98543  2.20%     -    0s
     0     0    1.98570    0   45    2.03000    1.98570  2.18%     -    0s
H    0     0                       2.0219471    1.98570  1.79%     -    0s
     0     0    1.98583    

c[1,12] 0.0
c[1,13] 0.0
c[1,14] 0.0
c[1,15] 0.0
c[1,16] 0.0
c[1,17] 0.0
c[1,18] 0.0
c[1,19] 0.0
c[1,20] 0.0
c[1,21] 0.0
c[1,22] 0.0
c[1,23] 0.0
c[1,24] 0.0
c[1,25] 0.0
c[1,26] 0.0
c[1,27] 0.0
c[1,28] 0.0
c[1,29] 0.0
c[1,30] 0.0
c[1,31] 0.0
c[1,32] 0.0
c[1,33] 0.0
c[1,34] 0.0
c[1,35] 0.0
c[1,36] 0.0
c[1,37] 0.0
c[1,38] 0.0
c[1,39] 0.0
c[1,40] 0.0
c[1,41] 0.0
c[1,42] 0.0
c[1,43] 0.0
c[1,44] 0.0
c[1,45] 0.0
c[1,46] 0.0
c[1,47] 0.0
c[1,48] 0.0
c[1,49] 0.0
c[1,50] 0.0
c[1,51] 0.0
c[1,52] 0.0
c[1,53] 0.0
c[1,54] 0.0
c[1,55] 0.0
c[1,56] 0.0
c[1,57] 0.0
c[1,58] 0.0
c[1,59] 0.0
c[1,60] 0.0
c[1,61] 0.0
c[1,62] 0.0
c[1,63] 0.0
c[1,64] 0.0
c[1,65] 0.0
c[1,66] 0.0
c[1,67] 0.0
c[1,68] 0.0
c[1,69] 0.0
c[1,70] 0.0
c[1,71] 0.0
c[1,72] 0.0
c[1,73] 1.0
c[1,74] 1.0
c[1,75] 0.0
c[1,76] 0.0
c[1,77] 0.0
c[1,78] 0.0
c[1,79] 0.0
c[1,80] 0.0
c[1,81] 0.0
c[1,82] 0.0
c[1,83] 0.0
c[1,84] 0.0
c[1,85] 1.0
c[1,86] 0.0
c[1,87] 1.0
c[1,88] 1.0
c[1,89] 0.0
c[1,90] 0.0
c[1,91] 0.0
c[1,92] 1.0
c[1,93] 1.0
c[1,94] 0.0
c[1,

In [ ]:
#Sample set
import random
import csv
dicti = {
  '00:00': 0,
  '00:15': 1,
  '00:30': 2
}
dicti.update([ ('00:45', 3) , ('01:00', 4) , ('01:15', 5) ,
                 ('01:30', 6),('01:45', 7),('02:00', 8),
                 ('02:15', 9),('02:30', 10),('02:45', 11),
                 ('03:00', 12),('03:15', 13),('03:30', 14) , 
                 ('03:45', 15) , ('04:00', 16) , ('04:15', 17),
                 ('04:30', 18),('04:45', 19),('05:00', 20),
                 ('05:15', 21),('05:30', 22),('05:45', 23),
                 ('06:00', 24),('06:15', 25) , ('06:30', 26),
                 ('06:45', 27),('07:00', 28),('07:15', 29),
                 ('07:30', 30),('07:45', 31),('08:00', 32),
                 ('08:15', 33),('08:30', 34) , ('08:45', 35) , 
                 ('09:00', 36) , ('09:15', 37),('09:30', 38),
                 ('09:45', 39),('10:00', 40),('10:15', 41),
                 ('10:30', 42),('10:45', 43),('11:00', 44),
                 ('11:15', 45) , ('11:30', 46),('11:45', 47),
                 ('12:00', 48),('12:15', 49),('12:30', 50),
                 ('12:45', 51),('13:00', 52),('13:15', 53),
                 ('13:30', 54) , ('13:45', 55) , ('14:00', 56) , 
                 ('14:15', 57),('14:30', 58),('14:45', 59),
                 ('15:00', 60),('15:15', 61),('15:30', 62),
                 ('15:45', 63),('16:00', 64),('16:15', 65) , 
                 ('16:30', 66),('16:45', 67),('17:00', 68),
                 ('17:15', 69),('17:30', 70),('17:45', 71),
                 ('18:00', 72),('18:15', 73),('18:30', 74) , 
                 ('18:45', 75) , ('19:00', 76) , ('19:15', 77),
                 ('19:30', 78),('19:45', 79),('20:00', 80),
                 ('20:15', 81),('20:30', 82),('20:45', 83),
                 ('21:00',84),('21:15', 85) , ('21:30', 86),
                 ('21:45', 87),('22:00', 88),('22:15', 89),
                 ('22:30', 90),('22:45', 91),('23:00', 92),
                 ('23:15', 93),('23:30', 94) , ('23:45', 95)])
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/Arr_Dep_EV.csv', newline='') as f:
    reader = csv.reader(f)
    #reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
    header = next(reader)
    y = []
    for row in reader :
            ydist = row[0]
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])

    ev1 =(random.choice(y))
    ev2 =(random.choice(y))
    print(ev1)
    print(ev2)
    for item,val in dicti.items():
        if item == ev1[1]:
            ev1_a = val
        elif item == ev1[2]:
            ev1_d = val
        elif item == ev2[1]:
            ev2_a = val
        elif item == ev2[2]:
            ev2_d = val
    if ev1_a < ev2_a and ev2_a < ev1_d and ev1_d < ev2_d:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev2 between ev1a and ev1d')
    elif ev2_a < ev1_a and ev1_a < ev2_d and ev2_d < ev1_d:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev1 between ev2a and ev2d')
    elif ev1_a < ev2_a and ev2_a < ev2_d and ev1_d < ev1_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev2 between ev1a and ev1d')
    elif ev2_a < ev1_a and ev1_a < ev1_d and ev2_d < ev2_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
    elif ev1_d < ev2_d and ev2_d < ev1_a and ev1_a < ev2_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev2 between ev1a and ev1d')
    elif ev2_d < ev1_d and ev1_d < ev2_a and ev2_a < ev1_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
#Distance to energy
import csv
from scipy import interpolate
distance =[]
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv') as file:
    reader = csv.reader(file)
    #header = next(reader)
    x = []
    for row in reader :
        xdist = float(row[0])
        xaDate = row[1]
        xdDate = row[2]
        x.append([xdist,xaDate,xdDate])
    #ev2 =(random.choice(x)) 
    for i in range(len(x)):
        #print(x[i][0])
        distance.append(x[i][0])
print(distance)
Distance = [0,100]    #Distance given /100 km
Energy = [0,17]  #Energy in kwh
f = interpolate.interp1d(Distance,Energy)
for i in range(len(distance)):
    evdistance = distance[i] 
    Energyeq = f(evdistance)
    print('Energy equivalent:',Energyeq)